In [ ]:
import fitz  # PyMuPDF for text and image extraction
import pdfplumber  # For table extraction
import os 
from VideoProcessor import VideoProcessor #this really isnt necessary, all this class is used for is setting up the connection to OpenAI so 
from openai import AzureOpenAI

azure_openai_key="20c7185be8f5419aa8c655e1ed514d4d"
azure_openai_endpoint="https://exploration-openai-platformteam.openai.azure.com/"

azure_service_region='eastus'
apiversion = "2024-02-01"
gptengine = "gpt-4o-mini"

openai_client = AzureOpenAI(
            api_key=azure_openai_key,
            api_version=apiversion,
            azure_endpoint=azure_openai_endpoint
        )

output_folder = r'pdf_to_image'
extracted_images=r"extracted_images"
pdf_path = r'pdfs'

processor = VideoProcessor(azure_openai_key="20c7185be8f5419aa8c655e1ed514d4d",
        azure_openai_endpoint="https://exploration-openai-platformteam.openai.azure.com/",
        azure_speech_key='6acf860ab894490b95b247537aeee4c2',
        azure_service_region='eastus',
        base_path=extracted_images,
        apiversion = "2024-02-01",
        gptengine = "gpt-4o")

#Submits a text only prompt to OpenAI API, this is really what the VideoProcessor should be too, instead of a class
def text_prompt(prompt):
    try:
            response = openai_client.chat.completions.create(
                model=gptengine,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt}
                        ]
                    }
                ],
                max_tokens=300
            )
            return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred while processing")
        return None


if not os.path.exists("extracted_images"):
    os.makedirs("extracted_images")

# List to hold all the extracted text
all_text = []

# Iterate through each page of the PDF
for pdf in os.listdir(pdf_path):
    doc = fitz.open(os.path.join(pdf_path, pdf))
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)  # Get a page
        
                                                                                                                                                                                        
        text = page.get_text("text")
        all_text.append(f"Page {page_num + 1}:\n{text}\n")
        print(page_num)

        #this is where we try to extract an table info
        try:
            with pdfplumber.open(pdf) as pdf_plumb:
            # Iterate through each page
                all_text.append(f"Tables on {page_num}:")
                tables = pdf_plumb.pages[page_num].extract_tables()
                for table in tables:
                    for row in table:
                        all_text.append(f"{row}")

        except:
            chartdescription = processor.analyze_image((f'{output_folder}\\page_{page_num + 101}.jpg'), f"this is an image of a pdf page, and here is the text extracted from the page: {text}, output the columns and rows labeled as so, but do not put the data, if the page contains no tables charts or graphs, Just say: --- ")
            print(f'chart description:{chartdescription}')
            prompt = f'this is the text extracted from a pdf: {text}, and this is the table information extracted from it {chartdescription}. with this information, describe the table in detail, if the page contains no discernable tables, Just say: --- '
            table_description = text_prompt(prompt)
            print(table_description)
            all_text.append(f"Page {page_num}, Tables: {table_description}")


        # Extract images
        image_list = page.get_images(full=True)  # Get all images on the page
        if image_list:
            for img_index, img in enumerate(image_list, start=1):
                # Extract the image bytes
                xref = img[0]  # Get image reference
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]
                
                # Save the image
                image_filename = f"extracted_images/page{page_num + 1}_img{img_index}.{image_ext}"
                imagedescription = processor.analyze_image((image_filename), "Describe this image in detail, keep things fairly spartan but accurate, this will be fed into a vectordb so avoid any formatting..")
                #print(image)
                all_text.append(f"Page {page_num}, Image {img_index}:\n{imagedescription}\n")
                with open(image_filename, "wb") as img_file:
                    img_file.write(image_bytes)

# Close the fitz document
doc.close()

# Save all extracted text to a file
with open(os.path.join('extracted_text', f'{pdf}_extracted.txt'), 'w', encoding='utf-8') as f:
    for page_text in all_text:
        f.write(page_text)

print(f"Text extraction complete. Images saved in 'extracted_images' folder. ")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
Text extraction complete. Images saved in 'extracted_images' folder. 
